In [1]:
import requests
import json

In [35]:
def lastfm_get(payload):

    headers = {'user-agent': 'naviak'}
    url = 'https://ws.audioscrobbler.com/2.0/'
    payload['api_key'] = ''
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [36]:
r = lastfm_get({
    'method': 'chart.getTopTracks'
})

js = {key: r.json()['tracks']['track'][0][key] for key in ('artist','name','url')}
js['artist'] = js['artist']['name']
jprint(js)

{
    "artist": "Mariah Carey",
    "name": "All I Want for Christmas Is You",
    "url": "https://www.last.fm/music/Mariah+Carey/_/All+I+Want+for+Christmas+Is+You"
}


In [6]:
import time
from IPython.core.display import clear_output

responses = []

page = 1
total_pages = 9

while page <= total_pages:
    payload = {
        'method': 'chart.getTopTracks',
        'limit': 500,
        'page': page
    }

    print("Requesting page {}/{}".format(page, total_pages))
    clear_output(wait = True)

    response = lastfm_get(payload)

    if response.status_code != 200:
        print(response.text)
        break

    page = int(response.json()['tracks']['@attr']['page'])
    #total_pages = int(response.json()['tracks']['@attr']['totalPages'])

    responses.append(response)

    if not getattr(response, 'from_cache', False):
        time.sleep(0.2)

    # increment the page number
    page += 1

Requesting page 9/9


In [30]:
def prettify(response):
    response_ = response.json()
    response_ = {key: response_['tracks']['track'][0][key] for key in ('artist','name','url')}
    response_['artist'] = response_['artist']['name']
    return response_

In [31]:
import pandas as pd

r0 = responses[0]
r0_df = pd.DataFrame([prettify(r0)])
r0_df.head()

,artist,name,url
0,bôa,Duvet,https://www.last.fm/music/b%C3%B4a/_/Duvet


In [34]:
frames = [pd.DataFrame([prettify(r)]) for r in responses]
artists = pd.concat(frames, ignore_index=True)
artists.head()

,artist,name,url
0,bôa,Duvet,https://www.last.fm/music/b%C3%B4a/_/Duvet
1,Mariah Carey,All I Want for Christmas Is You,https://www.last.fm/music/Mariah+Carey/_/All+I...
2,Slowdive,Alison,https://www.last.fm/music/Slowdive/_/Alison
3,Adele,Water Under The Bridge,https://www.last.fm/music/Adele/_/Water+Under+...
4,Frank Ocean,Sierra Leone,https://www.last.fm/music/Frank+Ocean/_/Sierra...
